# Notebook 4: Word Frequency Calculation

##### Please refer to the Python Requirements and Installation Guide pdf 

The function of this notebook is to calculate the frequencies of words that appear in each column that contains plain text in the Kununu reviews dataframe. This process is done for each company and for each year where reviews are written. The desired output is a JSON file which contains word frequency information per column, per year, and per company.

In [2]:
# Importing Packages
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
import json
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jupyter-
[nltk_data]     vinh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. Read & Concat CSV Files

In [3]:
def read_and_concat(path_to_csv_files: str):
    """
    This function reads in, concatenates, and prepares the translated csv files of each company's Kununu reviews.
        
    :param path_to_csv_files: path to folder containing the translated csv files

    :return: single dataframe which contains all the observations and features of each company's Kununu reviews              
    """
    csv_files = os.listdir(path_to_csv_files)
    df_list = []
    for i in csv_files:
        name = i.replace('_translated.csv', '')
        df = pd.read_csv(f'{path_to_csv_files}/{i}', sep = '\t')
        df = df.drop('Unnamed: 0', axis = 1)
        df.insert(0, 'company', [name] * df.shape[0])
        
        # Date separation (add separate `year` and `month` column)
        df.loc[:, 'review_date'] = pd.to_datetime(df.loc[:, 'review_date'])
        df.sort_values('review_date', ascending = False)
        df.insert(1, 'year', df['review_date'].dt.year)
        df.insert(2, 'month', df['review_date'].dt.month)
        df.pop('review_date')
        
        df_list.append(df)
        
    # Concatenating all the dataframes
    concat_df = pd.concat(df_list, axis = 0, ignore_index = True)
    
    return concat_df

# ---- Example code ----
# Make sure to not have a '/' character at the very end of the path. Please provide your own path to the folder of translated csv files.
# The files are read from the "translated_csvs_folder" folder'directory
df = read_and_concat('/home/jupyter-vinh/strategy/final_project/translated_csvs_folder')
# df.to_csv('/home/jupyter-vinh/concat_df.csv', index = False, header = True, sep = '\t')

# 2. Calculate Word Frequency

In [4]:
def extract_word_freq_yearly(df, *column_names):
    '''
    This function calculates the word frequency of text columns in a single dataframe.
    
    :param df: dataframe of company with which you would like to calculate the word frequency
    :param *column_names: arg variable which accepts a list of column names indicating which columns word frequency 
                          should be calculated
                          
    :return: a nested dictionary which supplies word frequency information hierarchically by year, column name, word, and frequency
    '''
    years = df['year'].unique()

    word_freq_per_year_and_col = dict()
    for c_name in column_names:
        
        word_freq_per_year = dict()
        for year in years:
            filtered_df = df.loc[df['year'] == year] # filter df by year
            filtered_df = filtered_df.loc[filtered_df[c_name].notnull()] # remove nan from column
            text_tokens = filtered_df[c_name].str.lower().str.split() # split text in each row into a list of tokens
            text_tokens = [i for i in text_tokens if i != []] # remove empty lists
            text_tokens = [[''.join(e for e in string if e.isalnum()) for string in lst] for lst in text_tokens] # remove punctuation, spaces, parentheses, etc.
            text_tokens = [[word for word in lst if not word in stopwords.words('english')] for lst in text_tokens] # remove stopwords
            all_words = [str(i) for lst in text_tokens for i in lst] # creates a single list of all the words in text_tokens
            all_words = [i for i in all_words if i] # remove empty strings
            
            dct = dict()
            for i in all_words:
                dct[i] = dct.get(i, 0) + 1
                
            word_freq_per_year[f'{year}'] = dct
    
        word_freq_per_year_and_col[c_name] = word_freq_per_year
    
    return word_freq_per_year_and_col

def company_bow(df):
    '''
    This function calculates the word frequency per company, per year, and per column using extract_word_freq_yearly as a helper
    function.
    
    :param df: a dataframe that contains the Kununu reviews of all the companies where you would like to calculate word frequency
    
    :return: a nested dictionary which supplies word frequency information hierarchically by company, year, column name, word, and frequency
    '''
    companies = df['company'].unique()
    all_bow = dict()
    for company in companies:
        filtered_df = df.loc[df['company'] == company]
        cols = list(filtered_df.filter(like = 'plain_text', axis = 1).columns)
        all_bow[company] = extract_word_freq_yearly(filtered_df, *cols)
    
    return all_bow

In [5]:
company_bow = company_bow(df)

In [6]:
# Code to store final word frequency dictionary as a json file.
# "company_bow.json" is found in the main directory - "Strategy Final Project"
# with open('/home/jupyter-vinh/company_bow.json', 'w') as f:
#     json.dump(company_bow, f)

# 3. Topic Modeling & Word Frequency

In [12]:
# Please provide your own path to the word dictionary.
# The word dictionary dervied through topic modelling can be found in 'Strategy Final Project'
gut_am_arbeitgeber = pd.read_excel('/home/jupyter-vinh/strategy/final_project/word_dictionary_final.xlsx', sheet_name = 'Sheet1', skiprows = 2)
schlecht_am_arbeitgeber = pd.read_excel('/home/jupyter-vinh/strategy/final_project/word_dictionary_final.xlsx', sheet_name = 'Sheet2', skiprows = 2)

def create_word_lists(df):
    '''
    This function creates a dictionary of words per topic given a dataframe.
    
    :param df: dataframe which contains dictionary of words which makes up different topics with a Kununu text review column
    
    :return: a dictionary where the key is a topic and the value is a list of words within that topic
    '''
    car_opp = list(df.loc[df['Topic 1: Career Opportunities'].notnull()]['Topic 1: Career Opportunities'])
    car_opp = [word.lower() for word in car_opp]
    
    remuneration = list(df.loc[df['Topic 2: Remuneration'].notnull()]['Topic 2: Remuneration'])
    remuneration = [word.lower() for word in remuneration]
    
    team_dyn = list(df.loc[df['Topic 3: Team Dynamic'].notnull()]['Topic 3: Team Dynamic'])
    team_dym = [word.lower() for word in team_dyn]
    
    return {'Career Opportunities': car_opp,
            'Remuneration': remuneration, 
            'Team Dynamic': team_dyn}
    
gut_topics_word_freq = create_word_lists(gut_am_arbeitgeber)
schlecht_topics_word_freq = create_word_lists(schlecht_am_arbeitgeber)

# 4. Time Series

In [13]:
def create_time_series_df(master_dict: dict, 
                          company_name: str, 
                          gut_topics: dict,
                          schlect_topics: dict):
    '''
    This function creates a dataframe which can then be used for time series analysis.
    
    :param master_dict: dictionary containing all word frequency information for each company
    :param company_name: name of company where you would like to perform time series analysis
    :param gut_topics: list of words for each topic in column `Gut am Arbeitgeber finde ich_plain_text`
    :param schlecht_topics: list of words for each topic in column `Gut am Arbeitgeber finde ich_plain_text`
    
    :return: A dataframe with four columns, topic, word_count, year, and feature. The word count column is a percentage
             indicating the percentage of words that appeared in a topic amongst all words in the feature (Kununu review column).
    '''
    
    df1_list = []
    years = master_dict[company_name]['Gut am Arbeitgeber finde ich_plain_text'].keys()
    for key, value in gut_topics.items():
        for year in years:
            word_count_single_year = master_dict[company_name]['Gut am Arbeitgeber finde ich_plain_text'][year]
            topic_word_count = dict.fromkeys(value, 0)
            for k in topic_word_count.keys():
                if k in word_count_single_year:
                    topic_word_count[k] = word_count_single_year[k]
            
            single_year = {'topic': [key],
                           'word_count': [sum(topic_word_count.values())],
                           'year': [int(year)],
                           'feature': ['Gut am Arbeitgeber finde ich_plain_text'.replace(' finde ich_plain_text', '')]
            }
            single_year_df = pd.DataFrame(single_year)
            df1_list.append(single_year_df)
    df1 = pd.concat(df1_list)
    
    df2_list = []
    years = master_dict[company_name]['Schlecht am Arbeitgeber finde ich_plain_text'].keys()
    for key, value in schlect_topics.items():
        for year in years:
            word_count_single_year = master_dict[company_name]['Schlecht am Arbeitgeber finde ich_plain_text'][year]
            topic_word_count = dict.fromkeys(value, 0)
            for k in topic_word_count.keys():
                if k in word_count_single_year:
                    topic_word_count[k] = word_count_single_year[k]
            
            single_year = {'topic': [key],
                           'word_count': [sum(topic_word_count.values())],
                           'year': [int(year)],
                           'feature': ['Schlecht am Arbeitgeber finde ich_plain_text'.replace(' finde ich_plain_text', '')]
            }
            single_year_df = pd.DataFrame(single_year)
            df2_list.append(single_year_df)
    df2 = pd.concat(df2_list)
    
    return pd.concat([df1, df2])

In [14]:
bechtle_visualization = create_time_series_df(company_bow, 'bechtle', gut_topics_word_freq, schlecht_topics_word_freq)

In [15]:
years = bechtle_visualization.year.unique()
for year in years:
    word_year_sum_gut = sum(company_bow['bechtle']['Gut am Arbeitgeber finde ich_plain_text'][str(year)].values())
    word_year_sum_schlecht = sum(company_bow['bechtle']['Schlecht am Arbeitgeber finde ich_plain_text'][str(year)].values())
    
    bechtle_visualization['word_count'] = np.where((bechtle_visualization.year == year) & (bechtle_visualization.feature == 'Gut am Arbeitgeber'),
                                                    bechtle_visualization.word_count / word_year_sum_gut,
                                                    bechtle_visualization.word_count)
    bechtle_visualization['word_count'] = np.where((bechtle_visualization.year == year) & (bechtle_visualization.feature == 'Schlecht am Arbeitgeber'),
                                                    bechtle_visualization.word_count / word_year_sum_schlecht,
                                                    bechtle_visualization.word_count)

In [16]:
# bechtle_visualization.to_csv('/home/jupyter-vinh/strategy/final_project/bechtle_visualization.csv')

In [17]:
sap_visualization = create_time_series_df(company_bow, 'sap', gut_topics_word_freq, schlecht_topics_word_freq)

In [18]:
years = sap_visualization.year.unique()
for year in years:
    word_year_sum_gut = sum(company_bow['sap']['Gut am Arbeitgeber finde ich_plain_text'][str(year)].values())
    word_year_sum_schlecht = sum(company_bow['sap']['Schlecht am Arbeitgeber finde ich_plain_text'][str(year)].values())
    
    sap_visualization['word_count'] = np.where((sap_visualization.year == year) & (sap_visualization.feature == 'Gut am Arbeitgeber'),
                                                sap_visualization.word_count / word_year_sum_gut,
                                                sap_visualization.word_count)
    sap_visualization['word_count'] = np.where((sap_visualization.year == year) & (sap_visualization.feature == 'Schlecht am Arbeitgeber'),
                                                sap_visualization.word_count / word_year_sum_schlecht,
                                                sap_visualization.word_count)

In [1]:
# sap_visualization.to_csv('/home/jupyter-vinh/strategy/final_project/sap_visualization.csv')

# 5. Word Cloud

In [22]:
def create_word_cloud_df(master_dict: dict,
                        company_name: str,
                        gut_topics: dict,
                        schlecht_topics: dict,
                        years: list):
    '''
    This function creates a dataframe which can then be used to create a word cloud.
    
    :param master_dict: dictionary containing all word frequency information for each company
    :param company_name: name of company where you would like to perform time series analysis
    :param gut_topics: list of words for each topic in column `Gut am Arbeitgeber finde ich_plain_text`
    :param schlecht_topics: list of words for each topic in column `Gut am Arbeitgeber finde ich_plain_text`
    :param years: a list which indicates which years you would like to look at to create the word cloud
    
    :return: A dataframe with three columns, feature, topic, text. The feature column indicates the Kununu review column. The topic column
             indicates the topic in the review column. The text column aggregates all the words in that topic in that column for the years specified.
    '''
    
    df = pd.DataFrame(columns = ['feature', 'topic', 'text'])
    
    for key, value in gut_topics.items():
        topic_word_count = dict.fromkeys(value, 0)
        for year in years:
            word_count_single_year = master_dict[company_name]['Gut am Arbeitgeber finde ich_plain_text'][year]
            for k in topic_word_count.keys():
                if k in word_count_single_year:
                    topic_word_count[k] += word_count_single_year[k]
        
        string = ''
        for _key, _value in topic_word_count.items():
            key_expand = ' '.join([_key] * _value) + ' '
            string += key_expand      
        
        row = pd.DataFrame({'feature': ['Gut am Arbeitgeber'],
                            'topic': [key],
                            'text': [string]})
        df = pd.concat([df, row])
    
    for key, value in schlecht_topics.items():
        topic_word_count = dict.fromkeys(value, 0)
        for year in years:
            word_count_single_year = master_dict[company_name]['Schlecht am Arbeitgeber finde ich_plain_text'][year]
            for k in topic_word_count.keys():
                if k in word_count_single_year:
                    topic_word_count[k] += word_count_single_year[k]
        
        string = ''
        for _key, _value in topic_word_count.items():
            key_expand = ' '.join([_key] * _value) + ' '
            string += key_expand      
        
        row = pd.DataFrame({'feature': ['Schlecht am Arbeitgeber'],
                            'topic': [key],
                            'text': [string]})
        df = pd.concat([df, row])

    return df

In [23]:
bechtle_wordcloud = create_word_cloud_df(company_bow, 'bechtle', gut_topics = gut_topics_word_freq, schlecht_topics = schlecht_topics_word_freq, years = ['2020', '2021', '2022'])
# bechtle_wordcloud.to_csv('/home/jupyter-vinh/strategy/final_project/bechtle_wordcloud.csv')
sap_wordcloud = create_word_cloud_df(company_bow, 'sap', gut_topics = gut_topics_word_freq, schlecht_topics = schlecht_topics_word_freq, years = ['2020', '2021', '2022'])
# sap_wordcloud.to_csv('/home/jupyter-vinh/strategy/final_project/sap_wordcloud.csv')


# Proportion Comparison

Please do not run this code, this is just data analysis work using the dataframe generated from the create_word_cloud_df function.

In [25]:
# bechtle_proportion = pd.read_csv('/home/jupyter-vinh/strategy/final_project/bechtle_visualization.csv')
# bechtle_proportion = bechtle_proportion.drop(['Unnamed: 0'], axis = 1)
# bechtle_proportion = bechtle_proportion.loc[bechtle_proportion['year'].isin([2020, 2021, 2022])]

# sap_proportion = pd.read_csv('/home/jupyter-vinh/strategy/final_project/sap_visualization.csv')
# sap_proportion = sap_proportion.drop(['Unnamed: 0'], axis = 1)
# sap_proportion = sap_proportion.loc[sap_proportion['year'].isin([2020, 2021, 2022])]

In [51]:
# bechtle_grouped = bechtle_proportion.groupby(['feature', 'topic'])[['word_count']].sum().reset_index()
# bechtle_grouped

,feature,topic,word_count
0,Gut am Arbeitgeber,Management,306
1,Gut am Arbeitgeber,Remuneration,75
2,Gut am Arbeitgeber,Work Environment,308
3,Schlecht am Arbeitgeber,Management,128
4,Schlecht am Arbeitgeber,Remuneration,77
5,Schlecht am Arbeitgeber,Work Environment,113


In [48]:
# sap_grouped = sap_proportion.groupby(['feature', 'topic'])[['word_count']].sum().reset_index()
# sap_grouped

,feature,topic,word_count
0,Gut am Arbeitgeber,Management,853
1,Gut am Arbeitgeber,Remuneration,275
2,Gut am Arbeitgeber,Work Environment,1003
3,Schlecht am Arbeitgeber,Management,133
4,Schlecht am Arbeitgeber,Remuneration,51
5,Schlecht am Arbeitgeber,Work Environment,82


#### End of Notebook